In [1]:
# imports
import json, os, time
import tmdbsimple as tmdb
import pandas as pd
from tqdm.notebook import tqdm_notebook

In [3]:
# load api key
with open('.secret/tmdb_api.json') as f:
    login = json.load(f)

login.keys()

dict_keys(['API_Key', 'API_Read_Access_Token'])

In [5]:
# set API_KEY variable
tmdb.API_KEY = login['API_Key']

In [6]:
# function to add certification to movie.info dict
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    
    # set movie id
    movie = tmdb.Movies(movie_id)

    # save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    # only get releases of the movie in the US
    for c in releases['countries']:
        
        # if country abbreviation == US
        if c['iso_3166_1'] == 'US':
            
            # save certification key into info dict
            info['certification'] = c['certification']
            
    return info

In [7]:
# test get_certification function
movie_id = 'tt0133093'
info = get_movie_with_rating(movie_id)
info['certification']

'R'

In [8]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file
    (filename). Adapted from: https://www.geeksforgeeks.org/
    append-to-json-file-using-python/"""
    
    with open(filename, 'r+') as file:
        
        # load existing data from file into dict
        file_data = json.load(file)
        
        # if both types of data are lists, extend
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
            
        # else, append
        else:
            file_data.append(new_data)
            
        # set current position at offset
        file.seek(0)
        
        # convert to json
        json.dump(file_data, file)

In [9]:
# load basics from Data folder
basics = pd.read_csv('Data/title_basics.csv.gz')

# check
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [11]:
# create list of years to extract from API
YEARS_TO_GET = [2000, 2001]
# create empty list to save errors
errors = []

In [13]:
# make folder for API call data
FOLDER = "DATA/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['ratings.csv.gz', 'title_basics.csv.gz', 'titles.csv.gz']

In [14]:
# outer loop controlled by YEARS_TO_GET

# set up progress bar for outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position = 0):
    
    # select JSON_FILE name to save results in progress
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # check if file already exists
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        
        # save an empty dict with "imdb_id" = 0 to new file
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id': 0}], f)
    
    # define df for current year
    df = basics.loc[basics['startYear'] == YEAR].copy()
    
    # save movie ids from df['tconst'] to list
    movie_ids = df['tconst'].copy()
    
    # load any existing data from json into previous_df
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any movies that already exist in JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
   
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc = f'Movies from {YEAR}',
                                 position = 1,
                                 leave = True):
        
        try:
            # retrieve data for movie_id
            temp = get_movie_with_rating(movie_id)
            
            # append/extend results to JSON_FILE
            write_json(temp, JSON_FILE)
            
            # pause to prevent overwhelming server with calls
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    # save year's results in csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                        compression = "gzip",
                        index = False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1448 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1569 [00:00<?, ?it/s]

In [15]:
# print number of errors encountered
print(f"- Total errors: {len(errors)}")

- Total errors: 443
